In [1]:
import tensorflow as tf
#import tensorflow_addons as tfa

import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import config as conf
import imageio

from IPython import display  

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats
import pickle

import AE_model as pcvae

In [44]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected = True)

import plotly.express as px
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import plotly.graph_objects as go

In [45]:
print(tf.__version__)

2.2.0


In [46]:
parent_dir = '/cavern/bihuayu/protein_DP/AE_symmetry/nodiag/realVal/'

In [47]:
def normalization(x):
    sz = tf.size(x['contact'])
    edge = tf.math.sqrt(tf.cast(sz, tf.float32))
    edge = tf.cast(edge, tf.int32)
    
    contact_img = x['contact']
    
    diag = tf.linalg.diag_part(x['contact'],k=(-1,1))
    x['diag'] = tf.linalg.diag(diag,k=(-1,1))
    x['contact'] = x['contact'] - x['diag']
    
    diag = tf.linalg.diag_part(contact_img, k = -1)
    diag2 = tf.linalg.diag(diag, k = -1)
    diag = tf.linalg.diag_part(contact_img, k = 1)
    diag3 = tf.linalg.diag(diag, k = 1)
    
    img = x['contact'] + 0.2 * (diag2 + diag3)
    
    img_norm = tf.expand_dims(img, -1)
    
    if edge > 600:
        img_norm = tf.image.resize_with_crop_or_pad(img_norm, 600, 600)
    elif edge < 8:
        img_norm = tf.image.resize_with_crop_or_pad(img_norm, 8, 8)
    elif edge % 8 != 0:
        mod = 8 - edge % 8
        img_norm = tf.image.resize_with_crop_or_pad(img_norm, edge+mod, edge+mod)
        
    x['contact'] = img_norm
    x['diag'] = img_norm
        
    return x

In [48]:
TRAIN_BUF = 60000
BATCH_SIZE = 1
TEST_BUF = 10000

import contact_map as cp
data = cp.contact_map()

val_varseq_norm = data.val_varseq.map(normalization)
it1_norm = data.it1.map(normalization)

val_varseq_set = val_varseq_norm.batch(BATCH_SIZE)
it1 = it1_norm.batch(BATCH_SIZE)

In [49]:
model = pcvae.CVAE(latent_dim = 50, mnist = False, units_list=[64,128,128])
model.compile(optimizer = model.optimizer, loss = model.compute_loss, metrics = ["accuracy"])
model.load_weights(parent_dir + 'cp-0010.ckpt')

CONTACT_MAP_log_norm


## Jacobian

In [18]:
img_lst = []
i = 0
for test_x in it1:
    if 'native' in test_x['name'][0].numpy().decode("utf-8"):
        sz = tf.shape(test_x['contact'])
        seq_len = sz[2]
        i = i + 1
        if i % 5 == 0:
            print(i, test_x['name'][0].numpy(), seq_len)
        if seq_len < 75:
            img_lst.append((test_x['name'], test_x['contact']))

1 b'1abv_native' tf.Tensor(104, shape=(), dtype=int32)
2 b'1af7__native' tf.Tensor(72, shape=(), dtype=int32)
3 b'1ah9_native' tf.Tensor(64, shape=(), dtype=int32)
4 b'1aoy_native' tf.Tensor(72, shape=(), dtype=int32)
5 b'1b4bAnative' tf.Tensor(72, shape=(), dtype=int32)
6 b'1b72Anative' tf.Tensor(56, shape=(), dtype=int32)
7 b'1bm8_native' tf.Tensor(104, shape=(), dtype=int32)
8 b'1bq9Anative' tf.Tensor(56, shape=(), dtype=int32)
9 b'1cewInative' tf.Tensor(112, shape=(), dtype=int32)
10 b'1cqkAnative' tf.Tensor(104, shape=(), dtype=int32)
11 b'1csp_native' tf.Tensor(72, shape=(), dtype=int32)
12 b'1cy5Anative' tf.Tensor(96, shape=(), dtype=int32)
13 b'1dcjA_native' tf.Tensor(80, shape=(), dtype=int32)
14 b'1di2A_native' tf.Tensor(72, shape=(), dtype=int32)
15 b'1dtjA_native' tf.Tensor(80, shape=(), dtype=int32)
16 b'1egxAnative' tf.Tensor(120, shape=(), dtype=int32)
17 b'1fadAnative' tf.Tensor(96, shape=(), dtype=int32)
18 b'1fo5Anative' tf.Tensor(88, shape=(), dtype=int32)
19 b'1g1cA

In [19]:
in_file = open("/cavern/bihuayu/protein_DP/AE_symmetry/nodiag/realVal/allJacPro.pkl", "wb")
pickle.dump(img_lst, in_file)
in_file.close()

In [50]:
@tf.function(experimental_relax_shapes=True)
def jac(p):
    with tf.GradientTape() as g:
        g.watch(p)
        y = model(p, training=False)
    jacobian = g.jacobian(y, p)
    return jacobian

In [40]:
jac_lst = []
k = 6
i = 0
for protein in img_lst:
    name = protein[0]
    p = protein[1]
    jacobian = jac(p)
    j = tf.squeeze(jacobian)
    
    J_shape = tf.shape(j)
    no_self_interaction = np.ones(J_shape)
    for m in range(J_shape[0]):
        for n in range(m,J_shape[0]):
            no_self_interaction[m,n,m,n] = 0
            no_self_interaction[m,n,n,m] = 0
            no_self_interaction[n,m,m,n] = 0
            no_self_interaction[n,m,n,m] = 0
    x = j*no_self_interaction
    top_values, top_indices = tf.nn.top_k(tf.reshape(x, (-1,)), k)
    top_list = []
    for max_id in top_indices:
        d1 = (J_shape[1]*J_shape[2]*J_shape[3])
        d2 = (J_shape[2]*J_shape[3])
        d3 = J_shape[3]
        ele_num = max_id + 1
        i1 = ele_num//d1 - 1 if ele_num%d1==0 else ele_num//d1
        i2 = (ele_num - d1*i1)//d2 - 1 if (ele_num - d1*i1)%d2 == 0 else (ele_num - d1*i1)//d2
        i3 = (ele_num - d1*i1 - d2*i2) // d3 - 1 if (ele_num - d1*i1 - d2*i2)%d3 == 0 else (ele_num - d1*i1 - d2*i2) // d3
        i4 = ele_num - d1*i1 - d2*i2 - i3*d3 - 1
        if i1 < i2:
            top_list.append((i1.numpy(), i2.numpy(), i3.numpy(), i4.numpy(), name.numpy()[0].decode("utf-8"), 'input:', p[0,i1,i2,0].numpy(), 'jac:', j[i1,i2,i3,i4].numpy()))
    jac_lst.append(top_list)
    i = i + 1
    if i % 5 == 0:
        print(i, name)

5 tf.Tensor([b'1b72Anative'], shape=(1,), dtype=string)
10 tf.Tensor([b'1itpAnative'], shape=(1,), dtype=string)
15 tf.Tensor([b'1ogwA_native'], shape=(1,), dtype=string)
20 tf.Tensor([b'1tfi_native'], shape=(1,), dtype=string)


In [44]:
len(jac_lst)

24

In [50]:
jac_lst

[[(39, 44, 44, 38, '1af7__native', 'input:', 1.01419, 'jac:', 0.6638674),
  (39, 44, 48, 36, '1af7__native', 'input:', 1.01419, 'jac:', 0.55937636),
  (39, 44, 46, 38, '1af7__native', 'input:', 1.01419, 'jac:', 0.5410222)],
 [(18, 25, 20, 25, '1ah9_native', 'input:', 0.698332, 'jac:', 0.75642395),
  (14, 39, 13, 39, '1ah9_native', 'input:', 0.0, 'jac:', 0.68199295),
  (29, 59, 56, 30, '1ah9_native', 'input:', 0.140701, 'jac:', 0.6779651)],
 [(10, 46, 46, 9, '1aoy_native', 'input:', 0.750786, 'jac:', 0.71841866),
  (14, 63, 14, 60, '1aoy_native', 'input:', 1.04686, 'jac:', 0.6861692),
  (46, 51, 43, 51, '1aoy_native', 'input:', 3.1305, 'jac:', 0.6453197)],
 [(38, 61, 35, 61, '1b4bAnative', 'input:', 1.35998, 'jac:', 0.54620826),
  (20, 21, 20, 22, '1b4bAnative', 'input:', 1.970982, 'jac:', 0.45639253),
  (6, 7, 6, 8, '1b4bAnative', 'input:', 1.6998739, 'jac:', 0.41052076)],
 [(6, 41, 3, 41, '1b72Anative', 'input:', 2.56592, 'jac:', 0.59448355),
  (43, 46, 43, 43, '1b72Anative', 'input:'

In [46]:
in_f = open("/cavern/bihuayu/protein_DP/AE_symmetry/nodiag/realVal/top3.pkl", "wb")
pickle.dump(img_lst, in_f)
in_f.close()

In [30]:
# for i in range(63):
#     tf.print(inputs[i, i+1])
## 2KDD_1_A contains 57 residues and padding is 7. It shows padding is 3 + 4. 
## So res_id = matrix_id - 2(counting from 1)